<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Reindeer_Ordering_Dec_2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://dmcommunity.org/wp-content/uploads/2017/11/reindeer.jpg)

[Here's](https://dmcommunity.org/challenge/challenge-dec-2017/) the link to the original challenge.

Santa always leaves plans for his elves to determine the order in which the reindeer will pull his sleigh. This year, for the European leg of his journey, his elves are working to the following schedule, which will form a single line of nine reindeer. Here are the rules:

1. Comet behind Rudolph, Prancer and Cupid
2. Blitzen behind Cupid
3. Blitzen in front of Donner, Vixen and Dancer
4. Cupid in front of Comet, Blitzen and Vixen
5. Donner behind Vixen, Dasher and Prancer
6. Rudolph behind Prancer
7. Rudolph in front of Donner, Dancer and Dasher
8. Vixen in front of Dancer and Comet
9. Dancer behind Donner, Rudolph and Blitzen
10. Prancer in front of Cupid, Donner and Blitzen
11. Dasher behind Prancer
12. Dasher in front of Vixen, Dancer and Blitzen
13. Donner behind Comet and Cupid
14. Cupid in front of Rudolph and Dancer
15. Vixen behind Rudolph, Prancer and Dasher.

In [2]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.opt import SolverStatus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from pyomo.contrib.latex_printer import latex_printer


In [85]:
model = ConcreteModel()

# Location
model.I = RangeSet(1,9)
# Reindeer
model.J = ['Dasher', 'Dancer', 'Prancer', 'Vixen', 'Comet', 'Cupid', 'Donner', 'Blitzen', 'Rudolph']

model.X = Var(model.I, model.J, within = Binary)

# One area per reindeer
def con0_rule(model,j):
  return sum(model.X[i,j] for i in model.I) == 1
model.con0 = Constraint(model.J, rule = con0_rule)

# One reindeer per area
def con01_rule(model,i):
  return sum(model.X[i,j] for j in model.J) == 1
model.con01 = Constraint(model.I, rule = con01_rule)

# Comet behind Rudolph, Prancer and Cupid
def con1_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Comet'])
  else:
    return model.X[i,j] == 0
model.con1 = Constraint(model.I, ['Rudolph','Prancer','Cupid'], rule = con1_rule)

# Blitzen behind Cupid

def con2_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Blitzen'])
  else:
    return model.X[i,j] == 0
model.con2 = Constraint(model.I, ['Cupid'], rule = con2_rule)

# Blitzen in front of Donner, Vixen and Dancer

def con3_rule(model,i,j):
  if i != 1:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(1,i-1) for k in ['Blitzen'])
  else:
    return model.X[i,j] == 0
model.con3 = Constraint(model.I, ['Donner', 'Vixen', 'Dancer'], rule = con3_rule)

# Cupid in front of Comet, Blitzen and Vixen

def con4_rule(model,i,j):
  if i != 1:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(1,i-1) for k in ['Cupid'])
  else:
    return model.X[i,j] == 0
model.con4 = Constraint(model.I, ['Comet', 'Blitzen', 'Vixen'], rule = con4_rule)

# Donner behind Vixen, Dasher and Prancer

def con5_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Donner'])
  else:
    return model.X[i,j] == 0
model.con5 = Constraint(model.I, ['Vixen','Dasher','Prancer'], rule = con5_rule)

# Rudolph behind Prancer

def con6_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Rudolph'])
  else:
    return model.X[i,j] == 0
model.con6 = Constraint(model.I, ['Prancer'], rule = con6_rule)

# Rudolph in front of Donner, Dancer and Dasher

def con7_rule(model,i,j):
  if i != 1:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(1,i-1) for k in ['Rudolph'])
  else:
    return model.X[i,j] == 0
model.con7 = Constraint(model.I, ['Donner', 'Dancer', 'Dasher'], rule = con7_rule)

# Vixen in front of Dancer and Comet

def con8_rule(model,i,j):
  if i != 1:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(1,i-1) for k in ['Vixen'])
  else:
    return model.X[i,j] == 0
model.con8 = Constraint(model.I, ['Dancer', 'Comet'], rule = con8_rule)

# Dancer behind Donner, Rudolph and Blitzen

def con9_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Dancer'])
  else:
    return model.X[i,j] == 0
model.con9 = Constraint(model.I, ['Donner', 'Rudolph', 'Blitzen'], rule = con9_rule)

# Prancer in front of Cupid, Donner and Blitzen

def con10_rule(model,i,j):
  if i != 1:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(1,i-1) for k in ['Prancer'])
  else:
    return model.X[i,j] == 0
model.con10 = Constraint(model.I, ['Cupid', 'Donner', 'Blitzen'], rule = con10_rule)

# Dasher behind Prancer

def con11_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Dasher'])
  else:
    return model.X[i,j] == 0
model.con11 = Constraint(model.I, ['Prancer'], rule = con11_rule)

# Dasher in front of Vixen, Dancer and Blitzen

def con12_rule(model,i,j):
  if i != 1:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(1,i-1) for k in ['Dasher'])
  else:
    return model.X[i,j] == 0
model.con12 = Constraint(model.I, ['Vixen', 'Dancer', 'Blitzen'], rule = con12_rule)

# Donner behind Comet and Cupid

def con13_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Donner'])
  else:
    return model.X[i,j] == 0
model.con13 = Constraint(model.I, ['Comet','Cupid'], rule = con13_rule)

# Cupid in front of Rudolph and Dancer

def con14_rule(model,i,j):
  if i != 1:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(1,i-1) for k in ['Cupid'])
  else:
    return model.X[i,j] == 0
model.con14 = Constraint(model.I, ['Rudolph', 'Dancer'], rule = con14_rule)

# Vixen behind Rudolph, Prancer and Dasher.

def con15_rule(model,i,j):
  if i != 9:
    return model.X[i,j] <= sum(model.X[x,k] for x in RangeSet(i+1,9) for k in ['Vixen'])
  else:
    return model.X[i,j] == 0
model.con15 = Constraint(model.I, ['Rudolph', 'Prancer', 'Dasher'], rule = con15_rule)

In [86]:
# Solve model
opt = SolverFactory('cbc')
result = opt.solve(model)

if (result.solver.status == SolverStatus.ok) and \
    (result.solver.termination_condition == TerminationCondition.optimal):
    # Do something when the solution in optimal and feasible
    print('Solution is Optimal')
elif (result.solver.termination_condition == TerminationCondition.infeasible):
    # Do something when model in infeasible
    print('Solution is Infeasible')
else:
        # Something else is wrong
    print("Solver Status:",  result.solver.status)

# Solve time
print('Solve Time: ', result.solver.wallclock_time)

Solution is Optimal
Solve Time:  0.02


In [89]:
x = model.X.extract_values()
sol = {}
for i in x.keys():
  if x[i]>0:
    sol[i] = x[i]

for i,j in sol.keys():
  print("{0} {1}".format(i,j))

1 Prancer
2 Cupid
3 Rudolph
4 Dasher
5 Blitzen
6 Vixen
7 Comet
8 Donner
9 Dancer
